In [218]:
import pandas as pd
import numpy as np
import sqlite3
import warnings
import re
from ipyvizzu import Data, Config, Style
from ipyvizzustory import Story, Slide, Step
from datetime import datetime
warnings.filterwarnings(action='ignore')

In [219]:
import functools # not required, but helps in production
def unpack_df_columns(func):
    """
    A general use decorator to unpack a df[subset] of columns
    into a function which expects the values at those columns
    as arguments
    """
    
    @functools.wraps(func)
    def _unpack_df_columns(*args, **kwargs):
        
        # args[0] is a pandas series equal in length as the 
        # df[subset] to which the apply function is applied 
        series = args[0]

        # series.values holds the number of arguments expected
        # by func and is os length len(df[subset].columns)
        return func(*series.values)

    return _unpack_df_columns

In [220]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("../db/hr.db")
df = pd.read_sql_query("SELECT * from base_info", con)

In [221]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [222]:
df.shape

(33463, 106)

In [223]:
df.head(1)

,level_0,열1,index,기준일자,회사,사업본부(지원),사업본부(공식),부문,부서,소속,직위,사번,Unnamed: 11,성명,조직총칭,주민번호,한문성명,생년월일,당사최초입사일자,당사입사일자,퇴직일자,퇴직사유,승진일자,승급일자,부서코드,조합비공제,직책보임일자,현직무일자,재직구분명,원소속조직총칭,원소속사업부,원소속부서,원소속부서명,원소속근무지,원소속직책,원소속직책보임일,고용형태,사원유형,직군,직렬,직무,직책,직급,그룹입사일자,성별,현자원구분,근무지,입사구분,채용경로,출신도,최종학교_학력,최종학교_졸업구분,최종학교_학위,최종학교_학교명,최종학교_전공,학사_학력,학사_졸업구분,학사_학위구분,학사_학교명,학사_전공,석사_학력,석사_졸업구분,석사_학위구분,석사_학교명,석사_전공,박사_학력,박사_졸업구분,박사_학위구분,박사_학교명,박사_전공,결혼여부,본적,우편번호,주소,집전화,사내전화,휴대폰,기본급,기초급,능력급,최고영어성적,최근영어성적,유효영어성적,인정영어성적,2018년상,2018년하,2019년상,2019년하,2020년상,2020년하,2021년상,2021년하,2022년상,2022년하,2018년상(등급),2018년하(등급),2019년상(등급),2019년하(등급),2020년상(등급),2020년하(등급),2021년상(등급),2021년하(등급),2022년상(등급),2022년하(등급),직간,EMAIL
0,0,0,0,t20210801,HCE,현대건설기계,현대건설기계,None,None,현대건설기계,자문역,A421197,None,송원종,현대건설기계,691023-1******,宋元琮,1969-10-23,2017-04-01,2021-01-01,None,None,2019-12-01,2019-12-01,None,N,None,2021-01-01,재직,현대건설기계,현대건설기계,None,None,분당사무소,None,None,비상근임원,퇴임임원,경영총괄(퇴임임원),경영총괄(퇴임임원),경영총괄(퇴임임원),None,상무급,2006-01-12,남성,민방위,분당사무소,특별채용,경력개별채용,부산광역시,대학교,졸업,학사,서울대,경영,대학교,졸업,학사,서울대,경영,None,None,None,None,None,None,None,None,None,None,기혼,부산부산진구,NaN,경기도 용인시 수지구 동천로 63번길 12 동천현대2차홈타운 206동 306호,None,6-6055,010-4646-3990,0.0,0.0,0.0,0.0,0.0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None,None,None,None,None,None,None,None,None,None,간접직,None


In [224]:
df["사원유형"].value_counts()

생산기술직      11931
사무기술직      11049
설계연구직       3626
설계기술직       2041
일반계약직       1517
연구직          663
전문직B         484
별정직          434
임원           339
전문직A         292
사무지원직        289
정규임원         276
별정직A         218
별정직B         130
퇴임임원          53
비상근임원         28
파견직           26
전문위원          23
사외이사/감사       22
전문직           11
계약임원           7
상근임원           4
Name: 사원유형, dtype: int64

In [225]:
columns = ['기준일자','회사','사업본부(지원)','부문','부서','소속','직위','사번','성명','성별','주민번호','생년월일','고용형태','사원유형','직군','직급','그룹입사일자','승급일자','입사구분']

In [226]:
def 고용형태_변환(df):
    회사 = df["회사"]
    고용형태 = df["고용형태"]
    사원유형 = df["사원유형"]
    
    if 회사 == "HDX" or 회사 == "HCE" or 회사 == "HCM":
        
        if 고용형태 == "정규임원"and 사원유형 == "임원":
            return "임원"
        elif 고용형태 == "정규임원"and 사원유형 == "전문위원":
            return "임원"
        elif 고용형태 == "전문위원"and 사원유형 == "전문위원":
            return "임원"
        elif 고용형태 == "계약임원"and 사원유형 == "임원":
            return "임원"
        
        elif 고용형태 == "상근임원"and 사원유형 == "퇴임임원":
            return "퇴임임원"
        elif 고용형태 == "비상근임원"and 사원유형 == "퇴임임원":
            return "퇴임임원"

        elif 고용형태 == "직원"and 사원유형 == "설계기술직":
            return "직원"
        elif 고용형태 == "직원"and 사원유형 == "연구직":
            return "직원"
        elif 고용형태 == "직원"and 사원유형 == "전문직":
            return "직원"
    
        elif 고용형태 == "일반계약직"and 사원유형 == "별정직":
            return "기간제"
        elif 고용형태 == "일반계약직"and 사원유형 == "사무기술직":
            return "기간제"
        elif 고용형태 == "일반계약직"and 사원유형 == "생산기술직":
            return "기간제"
        elif 고용형태 == "일반계약직"and 사원유형 == "설계기술직":
            return "기간제"
        elif 고용형태 == "일반계약직"and 사원유형 == "연구직":
            return "기간제"
        
        elif 고용형태 == "정년후계약직"and 사원유형 == "사무기술직":
            return "기간제"
        elif 고용형태 == "정년후계약직"and 사원유형 == "생산기술직":
            return "기간제"
        
        elif 고용형태 == "조건제계약직"and 사원유형 == "사무기술직":
            return "기간제"
        elif 고용형태 == "조건제계약직"and 사원유형 == "설계기술직":
            return "기간제"
        elif 고용형태 == "조건제계약직"and 사원유형 == "연구직":
            return "기간제"

        elif 고용형태 == "고용외국인"and 사원유형 == "사무기술직":
            return "기간제"
        elif 고용형태 == "파견후계약직"and 사원유형 == "별정직":
            return "기간제"
        
        elif 고용형태 == "조건제계약직"and 사원유형 == "연구직":
            return "기간제"
        else:
            return 고용형태
    else:
        return 고용형태


In [227]:
def 사원유형_변환(df):
    회사 = df["회사"]
    고용형태 = df["고용형태"]
    사원유형 = df["사원유형"]
    
    if 회사 == "HDX" or 회사 == "HCE" or 회사 == "HCM":
        
        if 고용형태 == "정규임원"and 사원유형 == "임원":
            return "정규임원"
        elif 고용형태 == "정규임원"and 사원유형 == "전문위원":
            return "전문위원"
        elif 고용형태 == "전문위원"and 사원유형 == "전문위원":
            return "전문위원"
        elif 고용형태 == "계약임원"and 사원유형 == "임원":
            return "계약임원"
        
        elif 고용형태 == "상근임원"and 사원유형 == "퇴임임원":
            return "상근임원"
        elif 고용형태 == "비상근임원"and 사원유형 == "퇴임임원":
            return "비상근임원"

        elif 고용형태 == "직원"and 사원유형 == "설계기술직":
            return "설계연구직"
        elif 고용형태 == "직원"and 사원유형 == "연구직":
            return "설계연구직"
        elif 고용형태 == "직원"and 사원유형 == "전문직":
            return "전문직A"
        elif 고용형태 == "직원"and 사원유형 == "사무지원직":
            return "사무지원/전문직B"
        
        elif 고용형태 == "일반계약직"and 사원유형 == "별정직":
            return "일반계약직"
        elif 고용형태 == "일반계약직"and 사원유형 == "사무기술직":
            return "일반계약직"
        elif 고용형태 == "일반계약직"and 사원유형 == "생산기술직":
            return "일반계약직"
        elif 고용형태 == "일반계약직"and 사원유형 == "설계기술직":
            return "일반계약직"
        elif 고용형태 == "일반계약직"and 사원유형 == "연구직":
            return "일반계약직"
        
        elif 고용형태 == "정년후계약직"and 사원유형 == "사무기술직":
            return "정년후계약직"
        elif 고용형태 == "정년후계약직"and 사원유형 == "생산기술직":
            return "정년후계약직"
        
        elif 고용형태 == "조건제계약직"and 사원유형 == "사무기술직":
            return "일반계약직"
        elif 고용형태 == "조건제계약직"and 사원유형 == "설계기술직":
            return "일반계약직"
        elif 고용형태 == "조건제계약직"and 사원유형 == "연구직":
            return "일반계약직"

        elif 고용형태 == "고용외국인"and 사원유형 == "사무기술직":
            return "일반계약직"
        elif 고용형태 == "파견후계약직"and 사원유형 == "별정직":
            return "파견후계약직"
        
        elif 고용형태 == "조건제계약직"and 사원유형 == "연구직":
            return "일반계약직"
        else:
            return 사원유형
        
    elif 회사 == "HDI":
        if 고용형태 == "직원"and 사원유형 == "전문직B":
            return "사무지원/전문직B"
        elif 고용형태 == "직원"and 사원유형 == "별정직A":
            return "별정직"
        elif 고용형태 == "직원"and 사원유형 == "별정직B":
            return "별정직"
        else:
            return 사원유형
        
    else:
        return 사원유형


In [228]:
@unpack_df_columns
def get_age(기준일자, birth_year):
    age = int(기준일자[1:5]) - int(birth_year[:4])
    return age

In [229]:
def add_age_range(age):
    return str(int(np.floor(age/10)*10))+"대"

In [230]:
def 직급간소화_변환(직급):
    if 직급 == "과장급":
        직급 = "HL3(1)"
    elif 직급 == "상무보급":
        직급 = "상무급"
    elif 직급 == "HL3":
        직급 = "HL3(1)"
    elif 직급 == "HL4":
        직급 = "HL3(2)"
    elif 직급 == "HL5":
        직급 = "HL3(3)"
    
    return 직급

In [231]:
def 현대인프라팀명추출(df):
    사업본부 = df["사업본부(지원)"]
    회사 = df["회사"]
    소속 = df["소속"]
    부서 = df["부서"]
    pattern1 = r'\b\w+(?:팀)'
    pattern2 = r'\b\w+(?:법인)'
    pattern3 = r'\b\w+(?:부)'

    if 회사 == "HDI":
        # "~부"를 추출하기 위해 "~본부"를 삭제하고 특수문제 &도 삭제한다.
        소속 = 소속.replace("본부", "").replace("&","").replace("/","")
        
        팀명 = re.findall(pattern1, 소속)   # 팀으로 끝나는 거 추출
        
        if len(팀명)  == 1:
            return f"{사업본부}_{팀명[0]}"
        
        else:   # 그런데.. 추출된게 없으면,
            팀명 = re.findall(pattern2, 소속)      # 부로 끝나는 거 추출
            
            if len(팀명) == 1:                 
                return f"{사업본부}_{팀명[0]}"     # 부로 추출된게 있으면... 리턴해라..
            
            else:
                팀명 = re.findall(pattern3, 소속)   # 부로도 추출된게 없으면.. 법인으로 끝나는 것 추출
                
                if len(팀명) == 1:           
                    return f"{사업본부}_{팀명[0]}"   # 법인으로 끝나는 것 추출된게 있으면 리턴
                else:
                    return None
    else:
        return 부서

In [232]:
def get_승급년도(승급일자):
    try:
        return 승급일자[:4]
    except:
        pass

In [233]:
@unpack_df_columns
def get_승급년차(기준일자, 승급년도):
    try:
        if 승급년도 is None:
            return 
        
        else:
            return int(기준일자[1:5]) - int(승급년도) + 1
    except:
        pass

## base DF1 생성

In [234]:
# 퇴임임원 자문역 날리기
df1 = df[columns]
df1.reset_index(drop=True, inplace=True)   # index 새로 부여
# df1["사원유형"].unique()

In [235]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33463 entries, 0 to 33462
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   기준일자      33463 non-null  object
 1   회사        33463 non-null  object
 2   사업본부(지원)  33458 non-null  object
 3   부문        11743 non-null  object
 4   부서        12812 non-null  object
 5   소속        33463 non-null  object
 6   직위        33463 non-null  object
 7   사번        33463 non-null  object
 8   성명        33463 non-null  object
 9   성별        33463 non-null  object
 10  주민번호      13168 non-null  object
 11  생년월일      33293 non-null  object
 12  고용형태      33463 non-null  object
 13  사원유형      33463 non-null  object
 14  직군        33463 non-null  object
 15  직급        24147 non-null  object
 16  그룹입사일자    33463 non-null  object
 17  승급일자      16120 non-null  object
 18  입사구분      13168 non-null  object
dtypes: object(19)
memory usage: 4.9+ MB


## 겸직발령인원 제거하기(HDI, HCE 각각)
- 제뉴인의 재정팀 및 내부회계 겸직자는 DB에서 날림

In [236]:
# HDI 겸직발령 임직원 인덱스 추출후 드랍
drop_df1 = df1.loc[(df1["사업본부(지원)"] == "현대제뉴인") & (df1["회사"] == "HDI")]
drop_index = drop_df1.index.tolist()
df1 = df1.drop(index=drop_index)

In [237]:
# 현건ㄴ기 겸직자 전처리 대상 사번
현건기겸직자사번리스트 = ['A524204',
 'A524205',
 'A524206',
 'A524207',
 'A524208',
 'A524209',
 'A524210',
 'A524211',
 'A524212',
 'A524213',
 'A524214',
 'A524215',
 'A524216',
 'A524217',
 'A524218',
 'A524219',
 'A524220',
 'A524221',
 'A524222',
 'A524223',
 'A524224',
 'A524225',
 'A524226',
 'A524227',
 'A524228',
 'A524229',
 'A524230',
 'A524231',
 'A524232',
 'A524233',
 'A524234',
 'A524235',
 'A524236',
 'A524237',
 'A524238',
 'A524239',
 'A524240',
 'A524241',
 'A524242',
 'A524243',
 'A524244',
 'A524245',
 'A524246',
 'A524247',
 'A524248',
 'A524249',
 'A524250',
 'A524251',
 'A524252',
 'A524253',
 'A524254',
 'A524255',
 'A524256',
 'A524257',
 'A524258',
 'A524259',
 'A524260',
 'A524307',
 'A523742',
 'A523767',
 'A523768',
 'A523743',
 'A523744',
 'A523745',
 'A523746',
 'A523748',
 'A523765',
 'A524456',
 'A524457']

In [238]:
'''
23년 1월 1일 현건기 기준명부상 겸직발령자 식별 불가
23년 2월 8일 발령부터 겸직자 부문명을 "겸직 제뉴인부문"으로 변경
이에 2월 8일 겸직자 사번 기준으로 23년 1월 1일 기준으로 대상자의 부문을 "겸직 제뉴인부문"으로 변경하고
부문명에 "겸직"이 들어간 대상자를 드랍하여 겸직자를 제거함
'''
def 현건기겸직자23년초부문변경(df):
    기준일자 = df['기준일자']
    사번 = df['사번']
    회사 = df['회사']
    부문 = df['부문']
    global 현건기겸직자사번리스트 

    if 기준일자 == "t20230101" and 회사 == "HCE" and 사번 in 현건기겸직자사번리스트:
        부문 = "겸직 제뉴인부문"
        return 부문
    else:
        return 부문

In [239]:
df1['부문'] = df1.apply(현건기겸직자23년초부문변경, axis=1)

In [240]:
# HCE 부문칼럼에 "겸직"이 있는 행 드랍
df1['부문'].fillna("empty_value", inplace=True)
drop = df1['부문'].str.contains("겸직")
df1 = df1[~drop]

In [241]:
# 제뉴인은 입사구분에 "계열회사 출입등록" 일괄 삭제
drop_df1 = df1.loc[(df1["회사"] == "HDX")&(df1["입사구분"] == "계열회사 출입등록")]
drop_index = drop_df1.index.tolist()
df1 = df1.drop(index=drop_index)

## 고용형태 및 사원유형 변환

In [242]:
# df1["사원유형"].value_counts()

In [243]:
# df1[df1["사원유형"] == "임원"]

In [244]:
df1["수정_고용형태"] = df1.apply(고용형태_변환, axis=1)
df1["수정_사원유형"] = df1.apply(사원유형_변환, axis=1)
df1["직급"] = df1["직급"].apply(직급간소화_변환)

df1["고용형태"] = df1["수정_고용형태"]
df1["사원유형"] = df1["수정_사원유형"]

In [245]:
# df1["고용형태"].value_counts()

In [246]:
df1["사원유형"].value_counts()

생산기술직        11349
사무기술직        10749
설계연구직         6140
일반계약직         2255
사무지원/전문직B      756
정규임원           569
별정직            471
전문직A           294
파견후계약직         230
비상근임원           80
계약임원            26
파견직             26
전문위원            22
사외이사/감사         22
정년후계약직          16
상근임원             5
Name: 사원유형, dtype: int64

In [247]:
# df1.info()

## 현대인프라팀명 추출

In [248]:
# HDI는 부서 칼럼값이 없어서 소속 정보에서 ~"팀"으로 끝나는 팀명을 추출하여 부서 칼럼값으로 넣기
df1["부서"] = df1.apply(현대인프라팀명추출, axis=1)  
# df1[df1["회사"]=="HDI"]["부서"].unique()

## 회사별 팀명 통합후 추출 (그룹핑 검토용)

In [249]:
# HDI_team = pd.DataFrame({"팀명": df1[df1["회사"]=="HDI"]["부서"].unique().tolist()})
# HDI_team["회사명"] = "HDI"

# HCE_team = pd.DataFrame({"팀명": df1[df1["회사"]=="HCE"]["부서"].unique().tolist()})
# HCE_team["회사명"] = "HCE"

# HDX_team = pd.DataFrame({"팀명": df1[df1["회사"]=="HDX"]["부서"].unique().tolist()})
# HDX_team["회사명"] = "HDX"

# total_team = pd.concat([HDI_team, HCE_team, HDX_team])
# len(total_team)


In [250]:
# total_team[total_team["회사명"] == "HDI"]["팀명"].tolist()

In [251]:
# 검토를 위해 CSV로 저장
# total_team.to_csv('total.csv', encoding="utf-8-sig")

## 팀 그룹핑 파일 불러오기

In [252]:
grouping_df = pd.read_csv("team_grouping.csv", encoding='UTF8')
grouping_df.tail()

,열1,부서,회사,Level1,Level2
448,95,컴팩추진지원팀,HDX,사업지원,재무/기획
449,96,정보보안팀,HDX,NaN,법무/보안
450,97,제도개선팀,HDX,NaN,HR/지원
451,98,Global HR팀,HDX,NaN,HR/지원
452,99,군산생산관리팀,HDX,산업차량,산차생산


In [253]:
# grouping_df["Level1"].unique()

In [254]:
# grouping_df["Level2"].unique()

In [255]:
# grouping_df[grouping_df["부서"] == "회계팀"]

In [256]:
# df1.head(2)

In [257]:
# df1["직급"].value_counts()

In [258]:
# df1["사원유형"].value_counts()

## DF2에 Left Join

In [259]:
def temp_key(val):
    key = "t"+str(hash(val))
    return key

In [260]:
def new_birthday(kf):
    global df2
    주민번호 = kf["주민번호"]
    생년월일 = kf["생년월일"]
    
    if 주민번호 != None:
        
        if 주민번호[0] == "0":

            temp = "20"+주민번호[:6]
            temp = temp[:4]+"-"+temp[4:6]+"-"+temp[6:8]
            temp = datetime.strptime(temp, '%Y-%m-%d')
        #     temp = temp.strftime('%Y-%m-%d')
            return temp

        else:
            temp = "19"+주민번호[:6]
            temp = temp[:4]+"-"+temp[4:6]+"-"+temp[6:8]
            temp = datetime.strptime(temp, '%Y-%m-%d')
        #     temp = temp.strftime('%Y-%m-%d')
            return temp
    
    else:
        return 생년월일
    

In [261]:
df2 = pd.merge(df1,grouping_df, on=['회사','부서'], how='left')
# 안쓰는 칼럼 날리고
df2.drop(['부문','열1','사업본부(지원)','소속', '직군'], axis=1, inplace=True)
# 팀 상위그룹핑 칼럼 추가
# df2.fillna("", inplace=True)
df2["Level1"].fillna("직할", inplace=True)
df2["Level2"].fillna("직할", inplace=True)
df2["그룹핑"] = df2["Level1"].str.cat(df2["Level2"],sep='_',na_rep="직할")
df2["새생년월일"] = df2.apply(new_birthday, axis=1)
df2["임시키"] = df2["새생년월일"].astype('str')+df2["성명"]
df2["임시키"] = df2["임시키"].apply(temp_key)
df2["연령"] = df2[["기준일자","새생년월일"]].astype('str').apply(get_age, axis=1)
df2["연령대"] = df2["연령"].apply(add_age_range)
df2["승급년도"] = df2["승급일자"].apply(get_승급년도)
df2["승급년차"] = df2[["기준일자","승급년도"]].apply(get_승급년차, axis=1)



In [262]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33010 entries, 0 to 33009
Data columns (total 26 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   기준일자     33010 non-null  object        
 1   회사       33010 non-null  object        
 2   부서       32471 non-null  object        
 3   직위       33010 non-null  object        
 4   사번       33010 non-null  object        
 5   성명       33010 non-null  object        
 6   성별       33010 non-null  object        
 7   주민번호     13083 non-null  object        
 8   생년월일     32904 non-null  object        
 9   고용형태     33010 non-null  object        
 10  사원유형     33010 non-null  object        
 11  직급       23826 non-null  object        
 12  그룹입사일자   33010 non-null  object        
 13  승급일자     15827 non-null  object        
 14  입사구분     13083 non-null  object        
 15  수정_고용형태  33010 non-null  object        
 16  수정_사원유형  33010 non-null  object        
 17  Level1   33010 non-null  object

In [263]:
df2.승급년도.unique()

array(['2019', '2021', '2020', '2015', '2017', '2013', '2018', '2011',
       '2012', '2014', '2016', '2006', '2010', '2008', '2009', '2005',
       '1999', '2007', '2022', '2023', '1997', None, '1900', '2002'],
      dtype=object)

In [264]:
df2.승급년차.unique()

array([  3.,   1.,   2.,   7.,   5.,   9.,   4.,  11.,  10.,   8.,   6.,
        16.,  12.,  14.,  13.,  17.,  23.,  15.,   0.,  24.,  18.,  19.,
        25.,  27.,  nan, 124.,  22.])

# 중간정리 DF3 (고용형태 : 임원 및 직원)

In [265]:
df3 = df2.loc[(df2["고용형태"] == "임원") | (df2["고용형태"] == "직원")]
# df3 = df3[df3["사원유형"] != "생산기술직"]
df3.shape

(30254, 26)

In [266]:
# df3["사원유형"].value_counts()

In [267]:
df3.head(2)

,기준일자,회사,부서,직위,사번,성명,성별,주민번호,생년월일,고용형태,사원유형,직급,그룹입사일자,승급일자,입사구분,수정_고용형태,수정_사원유형,Level1,Level2,그룹핑,새생년월일,임시키,연령,연령대,승급년도,승급년차
1,t20210801,HCE,None,기술자문,A510951,김태원,남성,580922-1******,1958-11-03,임원,계약임원,전무급,2021-07-05,2021-07-05,특별채용,임원,계약임원,직할,직할,직할_직할,1958-09-22,t149559542394686246,63,60대,2021,1.0
2,t20210801,HCE,None,자문,A510197,진재종,남성,670613-1******,None,임원,계약임원,전무급,2020-07-01,2020-07-01,특별채용,임원,계약임원,직할,직할,직할_직할,1967-06-13,t4634009041665087030,54,50대,2020,2.0


In [268]:
# len(df3["사번"].unique()), len(df3["임시키"].unique())

In [269]:
# df3[df3["성명"]=="허남섭"]

# 중간정리 DF4

In [270]:
df4 = df3[["임시키", "기준일자","회사", "고용형태", "사원유형", "직급","승급일자","승급년도","승급년차","부서","사번","성명","성별","Level1", "Level2","그룹핑","연령","연령대"]].reset_index(drop=True, inplace=False)
df4["rowid"] = df4.index
df4

,임시키,기준일자,회사,고용형태,사원유형,직급,승급일자,승급년도,승급년차,부서,사번,성명,성별,Level1,Level2,그룹핑,연령,연령대,rowid
0,t149559542394686246,t20210801,HCE,임원,계약임원,전무급,2021-07-05,2021,1.0,None,A510951,김태원,남성,직할,직할,직할_직할,63,60대,0
1,t4634009041665087030,t20210801,HCE,임원,계약임원,전무급,2020-07-01,2020,2.0,None,A510197,진재종,남성,직할,직할,직할_직할,54,50대,1
2,t4197481188501866770,t20210801,HCE,직원,사무기술직,HL3(3),2015-01-01,2015,7.0,Key Account팀,A460222,이경준,남성,영업,직할,영업_직할,55,50대,2
3,t4464187535224055059,t20210801,HCE,직원,사무기술직,HL3(3),2015-01-01,2015,7.0,글로벌교육센터,A321109,김성환,남성,영업,직할,영업_직할,54,50대,3
4,t-8837863997337252128,t20210801,HCE,직원,사무기술직,HL3(1),2019-01-01,2019,3.0,마케팅부,A488984,최영은,여성,영업,직할,영업_직할,36,30대,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30249,t-2649785603084945669,t20230101,HDI,직원,사무지원/전문직B,HL1,2023-01-01,2023,1.0,품질부문_품질기획팀,1925478,육애진,여성,직할,품질,직할_품질,30,30대,30249
30250,t9017451074972949249,t20230101,HDI,직원,사무지원/전문직B,HL1,2023-01-01,2023,1.0,품질부문_품질기획팀,1925657,김인영,여성,직할,품질,직할_품질,28,20대,30250
30251,t5703633505930366969,t20230101,HDI,직원,사무지원/전문직B,HL1,2023-01-01,2023,1.0,품질부문_품질기획팀,1926449,김가희,여성,직할,품질,직할_품질,31,30대,30251
30252,t1969343224930475568,t20230101,HDI,직원,사무기술직,HL3(3),2017-07-01,2017,7.0,품질부문_품질기획팀,1924619,권민재,남성,직할,품질,직할_품질,50,50대,30252


In [271]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30254 entries, 0 to 30253
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   임시키     30254 non-null  object 
 1   기준일자    30254 non-null  object 
 2   회사      30254 non-null  object 
 3   고용형태    30254 non-null  object 
 4   사원유형    30254 non-null  object 
 5   직급      22501 non-null  object 
 6   승급일자    14386 non-null  object 
 7   승급년도    14386 non-null  object 
 8   승급년차    14386 non-null  float64
 9   부서      29816 non-null  object 
 10  사번      30254 non-null  object 
 11  성명      30254 non-null  object 
 12  성별      30254 non-null  object 
 13  Level1  30254 non-null  object 
 14  Level2  30254 non-null  object 
 15  그룹핑     30254 non-null  object 
 16  연령      30254 non-null  int64  
 17  연령대     30254 non-null  object 
 18  rowid   30254 non-null  int64  
dtypes: float64(1), int64(2), object(16)
memory usage: 4.4+ MB


# 회사별 전입자 / 전출자 리스팅

In [272]:
직급정렬 = ["부회장급","사장급","부사장급","전무급","상무급","HL3(3)","HL3(2)","HL3(1)","HL2","HL1","S3","S2","S1","사원급" ]
사원유형정렬 = ["정규임원", "전문위원", "계약임원", "상근임원", "비상근임원", "퇴임임원", "설계연구직", "사무기술직", "생산기술직", "전문직A", "전문직","전문직B", "사무지원직", "별정직", "일반계약직", "정년후계약직", "파견후계약직", "고용외국인", "파견직"]

In [273]:
def 회사별유출인원(회사, before, after):
    global df4
    global 직급정렬
    global 사원유형정렬
    
    before_df = df4.loc[(df4["기준일자"] == before)&(df4["회사"] == 회사)]
    after_df = df4.loc[(df4["기준일자"] == after)&(df4["회사"] == 회사)]
    merged_df = before_df.merge(after_df, on='임시키',how='outer')
    유출자 = merged_df[merged_df["기준일자_y"].isnull()][["기준일자_x", "회사_x", "고용형태_x", "사원유형_x", "부서_x","직급_x", "성명_x", "임시키"]]
    
    유출자["사원유형_x"] = pd.Categorical(유출자['사원유형_x'], categories=사원유형정렬, ordered=True)
    유출자["직급_x"] = pd.Categorical(유출자['직급_x'], categories=직급정렬, ordered=True)

    유출자.sort_values(by=["사원유형_x","부서_x","직급_x"], inplace=True)
    return 유출자

In [274]:
r1 = 회사별유출인원("HDX","t20211001", "t20220101")
r1
len(r1["고용형태_x"])

7

In [275]:
def 회사별유입인원(회사, before, after):
    global df4
    global 직급정렬
    global 사원유형정렬
    
    before_df = df4.loc[(df4["기준일자"] == before)&(df4["회사"] == 회사)]
    after_df = df4.loc[(df4["기준일자"] == after)&(df4["회사"] == 회사)]
    merged_df = before_df.merge(after_df, on='임시키',how='outer')
    유입자 = merged_df[merged_df["기준일자_x"].isnull()][["기준일자_y", "회사_y", "고용형태_y", "사원유형_y", "부서_y","직급_y", "성명_y", "임시키"]]
    
    유입자["사원유형_y"] = pd.Categorical(유입자['사원유형_y'], categories=사원유형정렬, ordered=True)
    유입자["직급_y"] = pd.Categorical(유입자['직급_y'], categories=직급정렬, ordered=True)
    
    유입자.sort_values(by=["사원유형_y", "부서_y", "직급_y"], inplace=True, ascending=True)
    return 유입자

In [276]:
r2 =  회사별유입인원("HDI","t20220401", "t20220701")
r2["사원유형_y"].value_counts()
len(r2["고용형태_y"])

58

In [277]:
r2.head(2)

,기준일자_y,회사_y,고용형태_y,사원유형_y,부서_y,직급_y,성명_y,임시키
2665,t20220701,HDI,직원,설계연구직,건설기계사업본부_유압설계팀,HL3(2),배상진,t4656306026998993440
2666,t20220701,HDI,직원,설계연구직,건설기계사업본부_제어시스템개발팀,HL2,신대건,t3139008501788714577


# 간자(퇴사자 or 건기3사 외부 전적출자) 리스팅

In [278]:
def 간자리스팅(before, after):
    global df4
    before_df = df4[df4["기준일자"] == before]
    after_df = df4[df4["기준일자"] == after]
    merged_df = before_df.merge(after_df, on='임시키',how='outer')
    간자 = merged_df[merged_df["고용형태_y"].isnull()][["기준일자_x", "회사_x", "고용형태_x", "사원유형_x", "부서_x","직급_x", "성명_x", "임시키"]]
    간자 = 간자[간자["고용형태_x"]=="직원"]
    return 간자

In [279]:
간자 = 간자리스팅("t20211001", "t20220701")
간자["회사_x"].value_counts()

HDI    139
HCE     85
HDX     12
Name: 회사_x, dtype: int64

In [280]:
간자[간자["회사_x"]=="HDX"]

,기준일자_x,회사_x,고용형태_x,사원유형_x,부서_x,직급_x,성명_x,임시키
1477,t20211001,HDX,직원,사무기술직,기능품품질경영팀,HL3(3),이석희,t8323093922855533498
1483,t20211001,HDX,직원,설계연구직,유압기능품개발팀,HL3(3),권학순,t1103992255965265213
1547,t20211001,HDX,직원,사무기술직,기능품생산부,HL3(2),남병섭,t1780318246023589288
1553,t20211001,HDX,직원,사무기술직,기능품생산부,HL2,노기영,t-8508905146005264699
1579,t20211001,HDX,직원,사무기술직,FA팀,HL3(2),최진석,t-4712926987065027435
1587,t20211001,HDX,직원,사무기술직,회계팀,HL2,조성빈,t-4702645396694403740
1599,t20211001,HDX,직원,사무기술직,커뮤니케이션팀,HL3(2),이건희,t-1888068560954348083
1603,t20211001,HDX,직원,사무기술직,커뮤니케이션팀,HL3(1),이송,t-8087194617315373752
1620,t20211001,HDX,직원,사무기술직,기능품생산부,HL3(3),박영찬,t-1744961002638031327
1625,t20211001,HDX,직원,설계연구직,파워트레인개발팀,HL1,김태훈,t-7816363338394310233


# 온자 리스팅 (입사자, 건기3사 외부에서 전적입자)

In [281]:
def 온자리스팅(before, after):
    global df4
    before_df = df4[df4["기준일자"] == before]
    after_df = df4[df4["기준일자"] == after]
    merged_df = after_df.merge(before_df, on='임시키',how='outer')
    온자 = merged_df[merged_df["고용형태_y"].isnull()][["기준일자_x", "회사_x", "고용형태_x", "사원유형_x", "부서_x","직급_x", "성명_x", "임시키"]]
    온자 = 온자[온자["고용형태_x"]=="직원"]
    return 온자

In [282]:
온자 = 온자리스팅("t20220701", "t20221001")
온자["회사_x"].value_counts()

HDI    11
HDX     9
HCE     7
Name: 회사_x, dtype: int64

In [283]:
온자[온자["회사_x"]=="HDX"]

,기준일자_x,회사_x,고용형태_x,사원유형_x,부서_x,직급_x,성명_x,임시키
1299,t20221001,HDX,직원,사무기술직,EHS지원팀,HL3(2),이광명,t-8065344131066907668
1300,t20221001,HDX,직원,사무기술직,커뮤니케이션팀,HL2,이고은,t6674592543790923206
1333,t20221001,HDX,직원,설계연구직,산업차량제품설계팀,HL3(1),노정환,t-331763257128032991
1361,t20221001,HDX,직원,사무기술직,기능품생산부,HL3(1),도병훈,t5768882345924422008
1387,t20221001,HDX,직원,사무기술직,동력전장구매팀,HL2,박혜영,t1196438098621274960
1395,t20221001,HDX,직원,사무기술직,산업차량구매팀,HL3(1),이순진,t8751604168901432540
1530,t20221001,HDX,직원,사무지원/전문직B,커뮤니케이션팀,S1,도인혜,t-7719807959302095207
1686,t20221001,HDX,직원,사무지원/전문직B,회계팀,S1,최주연,t8049237915467032869
1693,t20221001,HDX,직원,사무기술직,인사지원팀,HL3(1),윤병곤,t-7595121564836711584


# 겸직임원 리스팅

In [284]:
def 겸직임원찾아내기(기준회사, 기준일자):
    global df4
    
    target_df = df4.loc[(df4["기준일자"] == 기준일자)&(df4["회사"] == 기준회사)&(df4["고용형태"]=="임원")]
    compare_df = df4.loc[(df4["기준일자"] == 기준일자)&(df4["회사"] != 기준회사)&(df4["고용형태"]=="임원")]

    겸직임원 = target_df.merge(compare_df, on='임시키',how='inner')  
    겸직임원["직급_x"] = pd.Categorical(겸직임원['직급_x'], categories=직급정렬, ordered=True)
    겸직임원.sort_values(by=["직급_x"], inplace=True)
    
    return 겸직임원[["rowid_x","기준일자_x","임시키", "직급_x", "성명_x", "회사_x", "회사_y"]]

In [285]:
기준일자들 = df.기준일자.unique().tolist()
회사들 = df.회사.unique().tolist()

In [286]:
겸직임원들 = []
for 회사 in 회사들:
    for 기준일자 in 기준일자들:
        temp_df = 겸직임원찾아내기(회사, 기준일자)
        겸직임원들.append(temp_df)


In [287]:
겸직임원_df = pd.concat(겸직임원들)
겸직임원_df.set_index('rowid_x', inplace=True)
겸직임원_df[겸직임원_df["성명_x"] == "최은록"]

,기준일자_x,임시키,직급_x,성명_x,회사_x,회사_y
rowid_x,,,,,,
6246,t20220401,t-1345097534201898341,상무급,최은록,HDX,HDI
7988,t20220701,t-1345097534201898341,상무급,최은록,HDX,HDI
9736,t20221001,t-1345097534201898341,상무급,최은록,HDX,HDI
21273,t20220401,t-1345097534201898341,상무급,최은록,HDI,HDX
23894,t20220701,t-1345097534201898341,상무급,최은록,HDI,HDX
26530,t20221001,t-1345097534201898341,상무급,최은록,HDI,HDX


In [288]:
# 겸직임원_df.groupby(["기준일자_x", "회사_x"])["임시키"].count()#.reset_index()

In [289]:
겸직임원날릴리스트 = 겸직임원_df[겸직임원_df["회사_x"] != "HDX"]
겸직임원날릴리스트.head(2)

,기준일자_x,임시키,직급_x,성명_x,회사_x,회사_y
rowid_x,,,,,,
3930,t20220101,t7932855782443827077,부사장급,김상웅,HCE,HDX
3463,t20220101,t-2704722446895536745,전무급,김인동,HCE,HDX


In [290]:
겸직임원날릴리스트 = 겸직임원날릴리스트.index.tolist()
# 겸직임원날릴리스트

In [291]:
def 겸직임원체크1(id):
    global 겸직임원날릴리스트
    
    if id in 겸직임원날릴리스트:
        return 1
    else:
        return 0

In [292]:
df4["겸직임원체크"] = df4['rowid'].apply(겸직임원체크1)

# DF5 (최종 결과)

In [293]:
df5 = df4[['임시키','기준일자','회사','고용형태','사원유형','직급','승급일자','승급년도','승급년차','성명','성별','Level1', 'Level2','그룹핑','연령','연령대','겸직임원체크']]
df5.head(3)

,임시키,기준일자,회사,고용형태,사원유형,직급,승급일자,승급년도,승급년차,성명,성별,Level1,Level2,그룹핑,연령,연령대,겸직임원체크
0,t149559542394686246,t20210801,HCE,임원,계약임원,전무급,2021-07-05,2021,1.0,김태원,남성,직할,직할,직할_직할,63,60대,0
1,t4634009041665087030,t20210801,HCE,임원,계약임원,전무급,2020-07-01,2020,2.0,진재종,남성,직할,직할,직할_직할,54,50대,0
2,t4197481188501866770,t20210801,HCE,직원,사무기술직,HL3(3),2015-01-01,2015,7.0,이경준,남성,영업,직할,영업_직할,55,50대,0


In [294]:
df5.drop("성명", axis=1)
df5["성명"] = '박보검'
df5.head(3)

,임시키,기준일자,회사,고용형태,사원유형,직급,승급일자,승급년도,승급년차,성명,성별,Level1,Level2,그룹핑,연령,연령대,겸직임원체크
0,t149559542394686246,t20210801,HCE,임원,계약임원,전무급,2021-07-05,2021,1.0,박보검,남성,직할,직할,직할_직할,63,60대,0
1,t4634009041665087030,t20210801,HCE,임원,계약임원,전무급,2020-07-01,2020,2.0,박보검,남성,직할,직할,직할_직할,54,50대,0
2,t4197481188501866770,t20210801,HCE,직원,사무기술직,HL3(3),2015-01-01,2015,7.0,박보검,남성,영업,직할,영업_직할,55,50대,0


In [295]:
# df5["승급년차"].value_counts()

In [296]:
df5.loc[(df5["회사"]=="HCE") & (df5["고용형태"] == "임원") & (df5["기준일자"] == "t20230101")]

,임시키,기준일자,회사,고용형태,사원유형,직급,승급일자,승급년도,승급년차,성명,성별,Level1,Level2,그룹핑,연령,연령대,겸직임원체크
10034,t6281199790706524479,t20230101,HCE,임원,정규임원,사장급,2022-11-02,2022,2.0,박보검,남성,직할,직할,직할_직할,63,60대,0
10035,t149559542394686246,t20230101,HCE,임원,계약임원,전무급,2021-07-05,2021,3.0,박보검,남성,직할,직할,직할_직할,65,60대,0
10041,t8626666440060965344,t20230101,HCE,임원,계약임원,전무급,2022-05-01,2022,2.0,박보검,남성,직할,직할,직할_직할,63,60대,0
10045,t4998690145228929751,t20230101,HCE,임원,정규임원,전무급,2020-12-01,2020,4.0,박보검,남성,직할,직할,직할_직할,54,50대,0
10046,t2583431202035860085,t20230101,HCE,임원,정규임원,상무급,2020-12-01,2020,4.0,박보검,남성,직할,직할,직할_직할,45,40대,0
10196,t1591808508133269675,t20230101,HCE,임원,정규임원,상무급,2021-11-01,2021,3.0,박보검,남성,직할,직할,직할_직할,50,50대,0
10281,t7191043501206784762,t20230101,HCE,임원,정규임원,상무급,2020-12-01,2020,4.0,박보검,남성,직할,직할,직할_직할,53,50대,0
10311,t1526619804323173833,t20230101,HCE,임원,정규임원,상무급,2021-11-01,2021,3.0,박보검,남성,직할,직할,직할_직할,50,50대,0
10316,t1059409424912931924,t20230101,HCE,임원,정규임원,상무급,2020-01-01,2020,4.0,박보검,남성,직할,직할,직할_직할,52,50대,0
10318,t-179720463622102466,t20230101,HCE,임원,정규임원,상무급,2018-01-01,2018,6.0,박보검,남성,직할,직할,직할_직할,54,50대,1


In [297]:
# df5.to_csv("result3.csv", encoding="utf-8-sig")

In [298]:
pdf1 = pd.pivot_table(df5[df5["회사"]=="HCE"], index=['고용형태','사원유형'], values=['임시키'], columns = ["기준일자"], aggfunc='count')
pdf1

임시키                                                    \
기준일자           t20210801 t20211001 t20220101 t20220401 t20220701 t20221001   
고용형태 사원유형                                                                    
임원   계약임원            3.0       3.0       3.0       3.0       3.0       2.0   
     전문위원            2.0       2.0       NaN       NaN       NaN       NaN   
     정규임원           26.0      27.0      26.0      26.0      26.0      26.0   
직원   별정직             NaN       NaN       NaN       NaN       1.0       1.0   
     사무기술직         504.0     548.0     452.0     464.0     482.0     472.0   
     사무지원/전문직B      22.0      31.0      26.0      26.0      26.0      28.0   
     생산기술직         533.0     535.0     508.0     512.0     513.0     515.0   
     설계연구직         319.0     323.0     246.0     246.0     240.0     239.0   

                          
기준일자           t20230101  
고용형태 사원유형                 
임원   계약임원            2.0  
     전문위원            NaN  
     정규임원           25.0  
직원   별정직             1.0  
     사무기술직         452.0  
     사무지원/전문직B      28.0  
     생산기술직         486.0  
     설계연구직         191.0

In [299]:
pdf1.reset_index()

고용형태       사원유형       임시키                                          \
기준일자                 t20210801 t20211001 t20220101 t20220401 t20220701   
0      임원       계약임원       3.0       3.0       3.0       3.0       3.0   
1      임원       전문위원       2.0       2.0       NaN       NaN       NaN   
2      임원       정규임원      26.0      27.0      26.0      26.0      26.0   
3      직원        별정직       NaN       NaN       NaN       NaN       1.0   
4      직원      사무기술직     504.0     548.0     452.0     464.0     482.0   
5      직원  사무지원/전문직B      22.0      31.0      26.0      26.0      26.0   
6      직원      생산기술직     533.0     535.0     508.0     512.0     513.0   
7      직원      설계연구직     319.0     323.0     246.0     246.0     240.0   

                          
기준일자 t20221001 t20230101  
0          2.0       2.0  
1          NaN       NaN  
2         26.0      25.0  
3          1.0       1.0  
4        472.0     452.0  
5         28.0      28.0  
6        515.0     486.0  
7        239.0     191.0

In [300]:
# pdf1.loc[(pdf1["사원유형"]=="사무기술직")&]

# 중간결과 저장

In [301]:
import pickle
with open ("pickle_df1.pickle", 'wb') as pickle_filename:
    pickle.dump(df5, pickle_filename)

In [302]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30254 entries, 0 to 30253
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   임시키     30254 non-null  object 
 1   기준일자    30254 non-null  object 
 2   회사      30254 non-null  object 
 3   고용형태    30254 non-null  object 
 4   사원유형    30254 non-null  object 
 5   직급      22501 non-null  object 
 6   승급일자    14386 non-null  object 
 7   승급년도    14386 non-null  object 
 8   승급년차    14386 non-null  float64
 9   성명      30254 non-null  object 
 10  성별      30254 non-null  object 
 11  Level1  30254 non-null  object 
 12  Level2  30254 non-null  object 
 13  그룹핑     30254 non-null  object 
 14  연령      30254 non-null  int64  
 15  연령대     30254 non-null  object 
 16  겸직임원체크  30254 non-null  int64  
dtypes: float64(1), int64(2), object(14)
memory usage: 3.9+ MB


In [303]:
df5 = pd.read_csv("../testfile1.csv")
df5

,year,grade,count
0,2019,사장,1
1,2019,부사장,1
2,2019,전무,3
3,2019,상무,7
4,2020,사장,2
5,2020,부사장,5
6,2020,전무,7
7,2020,상무,12
8,2021,사장,2
9,2021,부사장,4
